I am going to examine movies using our understanding of association rules. For this part, I implemented the apriori algorithm, and apply it to a movie rating dataset to find association rules of user-rate-movie behaviors. First, run the next cell to load the dataset we are going to use.

In [1]:
import numpy as np
user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))

array of user-movie matrix: shape (11743, 2)


In this dataset, there are two columns: the first column is the integer ids of users, and the second column is the integer ids of movies. Each row denotes that the user of given user id watched the movie of the given movie id. We are going to treat each user as a transaction, so you will need to collect all the movies that have been watched by a single user as a transaction.

Now, you need to implement the apriori algorithm and apply it to this dataset to find association rules of user movie-watching behaviors with **minimum support of 0.2** and **minimum confidence of 0.8**. We know there are many existing implementations of apriori online (check github for some good starting points). You are welcome to read existing codebases and let that inform your approach.

**Note: Do not copy-paste any existing code.**

**Note: We want your code to have sufficient comments to explain your steps, to show us that you really know what you are doing.**

**Note: You should add print statements to print out the intermediate steps of your method -- e.g., the size of the candidate set at each step of the method, the size of the filtered set, and any other important information you think will explain the process of the method.**

**Hint: If you implement your algorithm correctly, you should be able to see intermediate result as:**
- Candidates of length 1 count: 408
- After Pruning count: 21
- Candidates of length 2 count: 210
- After Pruning 2 count: 36
- Candidates of length 3 count: 55
- After Pruning 3 count: 12
- Candidates of length 4 count: 1
- After Pruning 4 count: 0

In [8]:
from collections import defaultdict
from itertools import combinations
import pandas as pd

# Load the movies.csv file
movies = pd.read_csv('movies.csv')

# Function to fetch movie names based on movie ids
def get_movie_names(movie_ids):
    """
    Fetches the movie names from the movies.csv file based on the given movie ids.

    Args:
        movie_ids (set): A set of movie ids.

    Returns:
        list: A list of movie names.
    """
    movie_names = []
    for movie_id in movie_ids:
        movie = movies[movies['movieId'] == movie_id]
        if not movie.empty:
            movie_names.append(movie['movie_name'].values[0])
    return movie_names

# Preprocess data into transactions
transactions = defaultdict(set)
for user_id, movie_id in user_movie_data:
    transactions[user_id].add(movie_id)

transactions = list(transactions.values())

# Function to calculate support
def calculate_support(candidate, transactions):
    """
    Calculates the support for a given candidate itemset.

    Args:
        candidate (frozenset): The candidate itemset.
        transactions (list): The list of transactions.

    Returns:
        float: The support for the candidate itemset.
    """
    count = sum(1 for transaction in transactions if candidate.issubset(transaction))
    return count / len(transactions)

# Function to generate candidates of a given length
def generate_candidates(prev_candidates, length):
    """
    Generates the candidate itemsets of a given length based on the previous frequent itemsets.

    Args:
        prev_candidates (set): The previous frequent itemsets.
        length (int): The desired length of the candidate itemsets.

    Returns:
        set: The set of candidate itemsets of the given length.
    """
    return set([frozenset(i.union(j)) for i in prev_candidates for j in prev_candidates if len(i.union(j)) == length])

# Apriori algorithm
def apriori(transactions, min_support, min_confidence):
    """
    Implements the Apriori algorithm to find frequent itemsets.

    Args:
        transactions (list): The list of transactions.
        min_support (float): The minimum support threshold.
        min_confidence (float): The minimum confidence threshold.

    Returns:
        list: The list of frequent itemsets.
    """
    # Initial candidate set (C1)
    candidates = set(frozenset([m]) for transaction in transactions for m in transaction)
    print(f"Candidates of length 1 count: {len(candidates)}")

    # Store frequent itemsets
    frequent_itemsets = []

    k = 2
    while candidates:
        # Calculate support for candidates and filter by min_support
        candidate_support = {c: calculate_support(c, transactions) for c in candidates}
        candidates = set(c for c, s in candidate_support.items() if s >= min_support)
        print(f"After Pruning {k-1} count: {len(candidates)}")

        if candidates:
            frequent_itemsets.extend(candidates)
            candidates = generate_candidates(candidates, k)
            print(f"Candidates of length {k} count: {len(candidates)}")
            k += 1

    return frequent_itemsets

    # Apply Apriori algorithm
frequent_itemsets = apriori(transactions, min_support=0.2, min_confidence=0.8)

Candidates of length 1 count: 408
After Pruning 1 count: 21
Candidates of length 2 count: 210
After Pruning 2 count: 36
Candidates of length 3 count: 123
After Pruning 3 count: 12
Candidates of length 4 count: 15
After Pruning 4 count: 0


Finally, print your final association rules in the following format:

**movie_name_1, movie_name_2, ... --> movie_name_k**

where the movie names can be fetched by joining the movieId with the file 'movies.csv'. For example, one rule that you should find is:

**Jurassic Park (1993), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)**

**Hint: if you implement the algorith correctly, you will find 14 rules in total.**

**Hint: You may need to use the Pandas library to load and process the movies.csv file, such as use pandas.read_csv() to load the data. https://pandas.pydata.org/pandas-docs/dev/user_guide/10min.html is a good place to learn the basics about Pandas.**

In [10]:
# Write your code to print out the rules
# Generate association rules from frequent itemsets
def generate_rules(frequent_itemsets, transactions, min_confidence):
    """
    Generates the association rules from the frequent itemsets.

    Args:
        frequent_itemsets (list): The list of frequent itemsets.
        transactions (list): The list of transactions.
        min_confidence (float): The minimum confidence threshold.

    Returns:
        list: The list of association rules.
    """
    rules = []
    for itemset in frequent_itemsets:
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent

                # Calculate confidence
                antecedent_support = calculate_support(antecedent, transactions)
                rule_support = calculate_support(itemset, transactions)
                confidence = rule_support / antecedent_support

                if confidence >= min_confidence:
                    antecedent_movies = get_movie_names(antecedent)
                    consequent_movies = get_movie_names(consequent)
                    rules.append((antecedent_movies, consequent_movies, rule_support, confidence))

    return rules

# Apply Apriori algorithm and generate rules
rules = generate_rules(frequent_itemsets, transactions, min_confidence=0.8)

# Print the association rules
for rule in rules:
    print(f"Rule: {rule[0]} -> {rule[1]}, Support: {rule[2]}, Confidence: {rule[3]}")

Rule: ['Godfather: Part II, The (1974)'] -> ['Godfather, The (1972)'], Support: 0.2388663967611336, Confidence: 0.8489208633093526
Rule: ['Jaws (1975)'] -> ['Star Wars: Episode IV - A New Hope (1977)'], Support: 0.23684210526315788, Confidence: 0.8068965517241379
Rule: ['Jurassic Park (1993)', 'Princess Bride, The (1987)'] -> ['Star Wars: Episode IV - A New Hope (1977)'], Support: 0.2165991902834008, Confidence: 0.9224137931034483
Rule: ['Star Wars: Episode IV - A New Hope (1977)', 'Princess Bride, The (1987)'] -> ['Back to the Future (1985)'], Support: 0.242914979757085, Confidence: 0.8
Rule: ['Princess Bride, The (1987)', 'Back to the Future (1985)'] -> ['Star Wars: Episode IV - A New Hope (1977)'], Support: 0.242914979757085, Confidence: 0.851063829787234
Rule: ['Jurassic Park (1993)', 'Back to the Future (1985)'] -> ['Star Wars: Episode IV - A New Hope (1977)'], Support: 0.2165991902834008, Confidence: 0.884297520661157
Rule: ['Saving Private Ryan (1998)', 'Back to the Future (1985